<a href="https://colab.research.google.com/github/dwwatson1/coronavirus_pandemic_playbook/blob/michael_work/md_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,780 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.co

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://cdc-main-database.s3.amazonaws.com/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"), header=True)

In [4]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|                           null|                  0|        

In [5]:
# Filter dates between March 2020 and December 2020
df = df.filter((df.case_month >= '2020-03') & (df.case_month <= '2020-12'))

In [6]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|            process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|                           null|                  0|           

In [7]:
# Filter for Maryland
df = df.filter((df.res_state == 'MD'))

In [8]:
# Drop unnecessary columns
df = df.drop('case_month', 'state_fips_code', 'res_county', 'county_fips_code', 'case_positive_specimen_interval', 'case_onset_interval', 'ethnicity', 'process', 'exposure_yn', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn', 'death_yn', 'underlying_conditions_yn')

In [9]:
# Replace "Missing" and "Unknown" with "NA"
df = df.replace(['Missing', 'Unknown'], 'NA')

In [10]:
df.show()

+---------+--------------+------+--------------+
|res_state|     age_group|   sex|          race|
+---------+--------------+------+--------------+
|       MD|18 to 49 years|Female|         White|
|       MD|18 to 49 years|Female|            NA|
|       MD|18 to 49 years|Female|         Black|
|       MD|  0 - 17 years|    NA|            NA|
|       MD|18 to 49 years|  Male|            NA|
|       MD|18 to 49 years|  Male|            NA|
|       MD|50 to 64 years|Female|            NA|
|       MD|18 to 49 years|Female|         White|
|       MD|18 to 49 years|  Male|         White|
|       MD|18 to 49 years|Female|            NA|
|       MD|18 to 49 years|Female|            NA|
|       MD|18 to 49 years|  Male|            NA|
|       MD|18 to 49 years|  Male|            NA|
|       MD|18 to 49 years|Female|            NA|
|       MD|18 to 49 years|  Male|         White|
|       MD|     65+ years|Female|Multiple/Other|
|       MD|18 to 49 years|Female|         White|
|       MD|  0 - 17 

In [11]:
df.groupby('res_state').count().orderBy('count', ascending=False).show()

+---------+------+
|res_state| count|
+---------+------+
|       MD|225814|
+---------+------+



In [12]:
df.groupby('age_group').count().orderBy('count', ascending=False).show()

+--------------+------+
|     age_group| count|
+--------------+------+
|18 to 49 years|123417|
|50 to 64 years| 46869|
|     65+ years| 32024|
|  0 - 17 years| 20670|
|            NA|  2834|
+--------------+------+



In [13]:
df.groupby('sex').count().orderBy('count', ascending=False).show()

+------+------+
|   sex| count|
+------+------+
|Female|114635|
|  Male|104308|
|    NA|  6871|
+------+------+



In [14]:
df.groupby('race').count().orderBy('count', ascending=False).show()

+--------------------+------+
|                race| count|
+--------------------+------+
|                  NA|136351|
|               White| 43793|
|               Black| 31577|
|      Multiple/Other| 13245|
|               Asian|   825|
|American Indian/A...|    23|
+--------------------+------+



In [16]:
df.toPandas().to_csv('maryland_data.csv', index=False)